In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Pandas

**Pandas** - библиотека, обеспечивающая высокопроизводительное лёгкое взаимодействие со структурами данных и их анализ. Основная структура данных в Pandas - **DataFrame**, представляющий собой двумерную таблицу (прямо как электронная таблица с названиями колонок и строчек).

Множество функций, доступных в Excel, доступны и с помощью Pandas, как например, создание сводных таблиц (pivot tables), вычисление одних колонок на основании других, отрисовка графиков и т.д. Также имеется возможность группировать строки по значениям столбца или джойнить (join tables) таблицы прямо как в SQL. Pandas также прекрасно справляется с временными рядами.

## Series объекты

Библиотека Pandas оперирует следующими полезными структурами данных

# apply / applymap / map

Для `DataFrame` есть две функции, позволяющих применить необходимую функцию для элементов таблицы - это методы `apply` и `applymap`.

Основная разница между ними в том, что функция `apply` применяется к элементам всей строки или столбца, в то время как метод `applymap` применяется поэлементно ко всей таблице. Название `applymap` пошло от того факта, что метод `map`, который есть у объекта `Series` применяется поэлементно ко всему объекту `Series`.  

In [7]:
df = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), index=list('zxcv'))
df

,b,d,e
z,-0.918327,-1.214585,-0.485532
x,-2.642260,-0.192416,1.266438
c,1.916433,1.594056,-0.793050
v,-1.181636,1.011530,0.529266


Применим такую функцию для строк и столбцов нашего `DataFrame`

In [8]:
f = lambda x: x.max() - x.min()

In [19]:
print(df.apply(f, axis=0))
print()
print(df.apply(f, axis=1))

b    4.558693
d    2.808640
e    2.059488
dtype: float64

z    0.729053
x    3.908698
c    2.709483
v    2.193167
dtype: float64


Однако, применить ту же функцию с помощью метода `applymap` мы уже не сможем, т.к. `max` и `min` в объекте `Series` есть. Но тут фукнция применяется **поэлементно**, а значит к элементам типа `float64`, и таких функций для этого типа уже нет.

Поэтому применим другую функцию с помощью метода `applymap`.

In [20]:
f = lambda x: '%.2f' % x

In [21]:
df.applymap(f)

,b,d,e
z,-0.92,-1.21,-0.49
x,-2.64,-0.19,1.27
c,1.92,1.59,-0.79
v,-1.18,1.01,0.53


Метод `applymap` имеет параметр `na_action`. Если он стоит в значение `ignore`, то он просто пропускает это значение и возвращает таблицу, в которой это же пустое значение есть. Однако, если параметр в значение `None`, то возникнет исключение, если в таблице есть пустой элемент.

In [29]:
df.loc['c', 'b'] = pd.NA
df

,b,d,e
z,-0.918327,-1.214585,-0.485532
x,-2.64226,-0.192416,1.266438
c,<NA>,1.594056,-0.793050
v,-1.181636,1.011530,0.529266


In [30]:
df.applymap(f, na_action='ignore')

,b,d,e
z,-0.92,-1.21,-0.49
x,-2.64,-0.19,1.27
c,<NA>,1.59,-0.79
v,-1.18,1.01,0.53


Ну и соответственно, функция `map`, схожая с `applymap`.

In [40]:
df['d'].map(f)

z    -1.21
x    -0.19
c     1.59
v     1.01
Name: d, dtype: object

**Резюмируя**: 
1) `map` определен только для `Series`, `applymap` - только для `DataFrame`, а `apply` - для обоих.  

2) `map` принимает как аргумент `dict`, `Series` или `callable` (это объект, который можно вызвать - он имеет метод `__call__`), в то время как `applymap` и `apply` принимают только `callable`.  

3) `map` применяется поэлементно к `Series`, `applymap` - поэлементно к `DataFrame`, а `apply` также поэлементно, но предполагает более сложные операции и агрегации, поэтому поведение и возвращаемое значение зависит от функции. 

4) `map` означает отображение одного элемента в другой, поэтому он отлично оптимизирован для этого (к примеру, `df['A'].map({1: 'a', 2: 'b', 3: 'c'})`). `applymap` прекрасно справляется с поэлементной трансформацией нескольких строк/столбцов (к примеру, `df[['A', 'B', 'C']].applymap(str.strip)`). `apply` может использовать любую функцию, которая не может быть векторизована (к примеру, `df['sentences'].apply(nltk.sent_tokenize)`). 

Ну и напоследок, следует отметить, что `map` позволяет использовать словари, а значит, это сильно ускоряет процесс трансформации и является прекрасным способом ускорить код. Поэтому, там, где можно использовать словари при подобных операциях, стоит это использовать.